In [1]:
import json
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import re
import os

In [2]:
def parse_json_metrics(file):
    with open(file, 'r') as file:
        content = json.load(file)

    vmaf_df = pd.json_normalize(content['vmaf'])
    psnr_df = pd.json_normalize(content['psnr'])
    ssim_df = pd.json_normalize(content['ssim'])

    return vmaf_df, psnr_df, ssim_df


In [9]:
vmaf_df, psnr_df, ssim_df = parse_json_metrics("D:\SystemFolders\Videos\\video_samples\\Witcher\\WITCHER3_libsvtav1_2M_vs_WITCHER3_metrics.txt")

In [11]:
vmaf_df

,integer_adm2,integer_adm_scale0,integer_adm_scale1,integer_adm_scale2,integer_adm_scale3,integer_motion2,integer_motion,integer_vif_scale0,integer_vif_scale1,integer_vif_scale2,integer_vif_scale3,vmaf,n
0,0.534035,0.778146,0.529147,0.425747,0.469661,0.000000,0.000000,0.032036,0.226611,0.329723,0.380818,0.000000,1
1,0.488459,0.765467,0.512352,0.396014,0.365702,4.556703,4.850279,0.033821,0.228902,0.329231,0.377029,0.000000,2
2,0.504987,0.769284,0.533751,0.432028,0.370280,4.556703,4.556703,0.035538,0.237934,0.341688,0.393300,0.000000,3
3,0.498819,0.770356,0.524599,0.431605,0.359855,4.678526,4.678526,0.036240,0.240391,0.342439,0.391059,0.000000,4
4,0.504153,0.763025,0.520460,0.409596,0.397634,4.135124,4.755467,0.036306,0.239067,0.341262,0.388537,0.000000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,0.650612,0.926059,0.769361,0.521963,0.394738,1.236370,1.241961,0.382612,0.584015,0.620902,0.616604,0.733239,3449
3449,0.647235,0.924272,0.722417,0.547869,0.388739,1.236370,1.236370,0.377111,0.583611,0.621740,0.617188,0.318678,3450
3450,0.740154,0.925368,0.808109,0.851614,0.400732,1.185908,1.243083,0.388231,0.590154,0.625027,0.619431,14.821258,3451
3451,0.697862,0.913772,0.883179,0.620213,0.396273,1.103269,1.185908,0.383881,0.586675,0.624200,0.618481,7.957204,3452


In [3]:

def extract_name(filename):
    pattern = r'(\w+)_lib((\w+[-\w]+)_(\d+)M)_vs_(\w+)_metrics\.txt'
    match = re.match(pattern, filename)
    if match:
        codec = match.group(2)
        bitrate = f"{match.group(3)}M"
        return codec, bitrate
    return None, None


In [4]:

def add_metrics_to_df(df, new_data, codec, bitrate):
    new_data['codec'] = codec
    new_data['bitrate'] = bitrate
    if df is None:
        return new_data
    else:
        return pd.concat([df, new_data], ignore_index=True)


In [5]:

def plot_metrics(df, metrics_to_plot, metric_name):
    fig = make_subplots(rows=1, cols=1, subplot_titles=(metrics_to_plot[metric_name],))
    for bitrate in df['bitrate'].unique():
        bitrate_df = df[df['bitrate'] == bitrate]
        for column in bitrate_df.columns:
            if column not in ['n', 'codec', 'bitrate']:
                for codec in bitrate_df['codec'].unique():
                    codec_df = bitrate_df[bitrate_df['codec'] == codec]
                    x_values = codec_df['n'] if 'n' in codec_df.columns else list(range(1, len(codec_df) + 1))
                    y_values = codec_df[column]
                    fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name=f"{codec} - {column}"), row=1, col=1)

    fig.update_layout(title=f"{metrics_to_plot[metric_name]}")
    fig.show()


In [6]:

def plot_metrics_from_files(dir, metrics_to_plot):
    metrics = {
        'vmaf': 'VMAF Metrics',
        'psnr': 'PSNR Metrics',
        'ssim': 'SSIM Metrics'
    }

    vmaf_df = None
    psnr_df = None
    ssim_df = None

    for filename in os.listdir(dir):
        if filename.endswith('metrics.txt'):
            codec, bitrate = extract_name(filename)
            if codec:
                vmaf_data, psnr_data, ssim_data = parse_json_metrics(os.path.join(dir, filename))
                vmaf_df = add_metrics_to_df(vmaf_df, vmaf_data, codec, bitrate)
                psnr_df = add_metrics_to_df(psnr_df, psnr_data, codec, bitrate)
                ssim_df = add_metrics_to_df(ssim_df, ssim_data, codec, bitrate)

    if 'vmaf' in metrics_to_plot:
        plot_metrics(vmaf_df, metrics, 'vmaf')
    if 'psnr' in metrics_to_plot:
        plot_metrics(psnr_df, metrics, 'psnr')
    if 'ssim' in metrics_to_plot:
        plot_metrics(ssim_df, metrics, 'ssim')


In [7]:


plot_metrics_from_files("D:\\SystemFolders\\Videos\\video_samples", ['vmaf', 'psnr'])

TypeError: 'NoneType' object is not subscriptable